In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.utils import resample
import matplotlib.ticker as ticker
%matplotlib inline 
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize
from nltk.corpus import stopwords
import warnings
from sklearn.utils import resample
from ast import literal_eval
import re
warnings.filterwarnings('ignore')
print('pandas:'+pd.__version__)
print('seaborn:'+sns.__version__)  

pandas:1.0.5
seaborn:0.10.1


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Importando os dados


In [3]:
tmdb = pd.read_csv('../../../dados/tmdb_5000_movies.csv')

In [4]:
tmdb.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [5]:
tmdb['id'] = tmdb['id'].astype('int')

# Classificação de Gêneros

## Preparação dos dados

### Tranformando a lista JSON de Gêneros em um array

In [79]:
tmdb['genres'] = tmdb['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
tmdb.head(3)

ValueError: malformed node or string: ['Action', 'Adventure', 'Fantasy', 'Science Fiction']

### Tranformando a lista JSON de keywords em um array

In [80]:
tmdb['keywords2'] = tmdb['keywords'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
tmdb.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,keywords2
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[culture clash, future, space war, space colon..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[ocean, drug abuse, exotic island, east india ..."
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[spy, based on novel, secret agent, sequel, mi..."


### Criação de 3 datasets para testes de algoritmos

In [81]:
title_df = tmdb[["overview","genres","title", "keywords2"]]
filmes_balanceados = title_df.copy(deep=True)

In [82]:
lst_col = 'genres'

filmes_balanceados2 = pd.DataFrame({
      col:np.repeat(filmes_balanceados[col].values, filmes_balanceados[lst_col].str.len())
      for col in filmes_balanceados.columns.drop(lst_col)}
    ).assign(**{lst_col:np.concatenate(filmes_balanceados[lst_col].values)})[filmes_balanceados.columns]

In [83]:
stre = 'Science  Fiction'
stre.split(',')

['Science  Fiction']

In [84]:
filmes_balanceados2['genres2'] = filmes_balanceados2['genres'].map(lambda x: x.split(','))
filmes_balanceados2['genres2'] 

0                 [Action]
1              [Adventure]
2                [Fantasy]
3        [Science Fiction]
4              [Adventure]
               ...        
12155             [Comedy]
12156              [Drama]
12157            [Romance]
12158           [TV Movie]
12159        [Documentary]
Name: genres2, Length: 12160, dtype: object

In [85]:
title_df.head(3)

,overview,genres,title,keywords2
0,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]",Avatar,"[culture clash, future, space war, space colon..."
1,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,"[ocean, drug abuse, exotic island, east india ..."
2,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]",Spectre,"[spy, based on novel, secret agent, sequel, mi..."


In [86]:
filmes_balanceados2.tail(3)

,overview,genres,title,keywords2,genres2
12157,"""Signed, Sealed, Delivered"" introduces a dedic...",Romance,"Signed, Sealed, Delivered","[date, love at first sight, narration, investi...",[Romance]
12158,"""Signed, Sealed, Delivered"" introduces a dedic...",TV Movie,"Signed, Sealed, Delivered","[date, love at first sight, narration, investi...",[TV Movie]
12159,Ever since the second grade when he first saw ...,Documentary,My Date with Drew,"[obsession, camcorder, crush, dream girl]",[Documentary]


In [87]:
filmes_balanceados2['genres'].value_counts()

Drama              2297
Comedy             1722
Thriller           1274
Action             1154
Romance             894
Adventure           790
Crime               696
Science Fiction     535
Horror              519
Family              513
Fantasy             424
Mystery             348
Animation           234
History             197
Music               185
War                 144
Documentary         110
Western              82
Foreign              34
TV Movie              8
Name: genres, dtype: int64

#### Separando cada gênero em dataset diferente para poder fazer resample dos dados - Tratamento de classes desbalanceadas

In [88]:
df_Drama = filmes_balanceados2[filmes_balanceados2['genres']=='Drama']
df_Comedy  = filmes_balanceados2[filmes_balanceados2['genres']=='Comedy']
df_Thriller   = filmes_balanceados2[filmes_balanceados2['genres']=='Thriller']
df_Action  = filmes_balanceados2[filmes_balanceados2['genres']=='Action']
df_Romance   = filmes_balanceados2[filmes_balanceados2['genres']=='Romance']
df_Adventure = filmes_balanceados2[filmes_balanceados2['genres']=='Adventure']
df_Crime    = filmes_balanceados2[filmes_balanceados2['genres']=='Crime']
df_Science_Fiction = filmes_balanceados2[filmes_balanceados2['genres']=='Science Fiction']
df_Horror     = filmes_balanceados2[filmes_balanceados2['genres']=='Horror']
df_Family           = filmes_balanceados2[filmes_balanceados2['genres']=='Family']
df_Fantasy    = filmes_balanceados2[filmes_balanceados2['genres']=='Fantasy']
df_Mystery      = filmes_balanceados2[filmes_balanceados2['genres']=='Mystery']
df_Animation   = filmes_balanceados2[filmes_balanceados2['genres']=='Animation']
df_History       = filmes_balanceados2[filmes_balanceados2['genres']=='History']
df_Music         = filmes_balanceados2[filmes_balanceados2['genres']=='Music']
df_War        = filmes_balanceados2[filmes_balanceados2['genres']=='War']
df_Documentary  = filmes_balanceados2[filmes_balanceados2['genres']=='Documentary']
df_Western     = filmes_balanceados2[filmes_balanceados2['genres']=='Western']
df_Foreign     = filmes_balanceados2[filmes_balanceados2['genres']=='Foreign']
df_TV_Movie  = filmes_balanceados2[filmes_balanceados2['genres']=='TV Movie']

In [89]:
 
df_Comedy  = resample(df_Comedy, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Thriller   = resample(df_Thriller, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Action  = resample(df_Action, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Romance   = resample(df_Romance, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Adventure= resample(df_Adventure, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Crime   = resample(df_Crime, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Science_Fiction = resample(df_Science_Fiction, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Horror     = resample(df_Horror, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Family          = resample(df_Family, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Fantasy   = resample(df_Fantasy, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Mystery      = resample(df_Mystery, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Animation  = resample(df_Animation, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_History    = resample(df_History, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Music      = resample(df_Music, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_War       = resample(df_War, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Documentary= resample(df_Documentary, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Western    = resample(df_Western, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_Foreign   = resample(df_Foreign, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)
df_TV_Movie = resample(df_TV_Movie, 
                                 replace=True,     # sample with replacement
                                 n_samples=2297,    # to match majority class
                                 random_state=123)

In [90]:
df_War.shape

(2297, 5)

In [91]:
df_upsampled = pd.concat([df_Drama, df_Comedy], axis=0 )
df_upsampled = pd.concat([df_upsampled, df_Thriller ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Action ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Romance ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Crime ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Science_Fiction ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Horror ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Family ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Fantasy ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Mystery ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Animation ]
                         )
df_upsampled = pd.concat([df_upsampled, df_History ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Music ]
                         )
df_upsampled = pd.concat([df_upsampled, df_War ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Documentary ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Western ]
                         )
df_upsampled = pd.concat([df_upsampled, df_Foreign ]
                         )
df_upsampled = pd.concat([df_upsampled, df_TV_Movie ]
                         )

In [92]:
df_upsampled.shape

(43643, 5)

In [93]:
df_upsampled.reset_index()

,index,overview,genres,title,keywords2,genres2
0,12,Following the death of District Attorney Harve...,Drama,The Dark Knight Rises,"[dc comics, crime fighter, terrorist, secret i...",[Drama]
1,74,"In 1933 New York, an overly ambitious movie pr...",Drama,King Kong,"[film business, screenplay, show business, fil...",[Drama]
2,76,"84 years later, a 101-year-old woman named Ros...",Drama,Titanic,"[shipwreck, iceberg, ship, panic, titanic, oce...",[Drama]
3,139,Life for former United Nations investigator Ge...,Drama,World War Z,"[dystopia, apocalypse, zombie, nuclear weapons...",[Drama]
4,153,An adaptation of F. Scott Fitzgerald's Long Is...,Drama,The Great Gatsby,"[based on novel, infidelity, obsession, hope, 3d]",[Drama]
...,...,...,...,...,...,...
43638,12158,"""Signed, Sealed, Delivered"" introduces a dedic...",TV Movie,"Signed, Sealed, Delivered","[date, love at first sight, narration, investi...",[TV Movie]
43639,9934,"Troy (Zac Efron), the popular captain of the b...",TV Movie,High School Musical,"[becoming an adult, musical, music, high schoo...",[TV Movie]
43640,8903,A young woman finds out that her mother worked...,TV Movie,Restless,"[spy, war, british secret service]",[TV Movie]
43641,10413,The continued Westward journey of settlers Mis...,TV Movie,Love's Abiding Joy,[],[TV Movie]


In [94]:
df_upsampled['genres'].value_counts()

Comedy             2297
Science Fiction    2297
Romance            2297
War                2297
Fantasy            2297
Animation          2297
TV Movie           2297
History            2297
Foreign            2297
Horror             2297
Crime              2297
Documentary        2297
Thriller           2297
Mystery            2297
Family             2297
Western            2297
Drama              2297
Action             2297
Music              2297
Name: genres, dtype: int64

In [95]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [96]:
punctuation = """!()-[]{};:'"\, <>./?@#$%^&*_~"""

words = stopwords.words("english")
lemma = nltk.stem.WordNetLemmatizer()
def pre_process(text):
    text = str(text)
    remove_hyperlink = re.sub('http://\S+|https://\S+', '', text)
    for elements in remove_hyperlink:
        if elements in punctuation:
            remove_hyperlink = remove_hyperlink.replace(elements, " ")
    tokens = word_tokenize(remove_hyperlink)
    remove_words = [word for word in tokens if not word in words]
    text = [lemma.lemmatize(word) for word in remove_words]
    joined_words = " ".join(text)
    return joined_words

#### Preprocessamento para retirada de strings não úteis para o machine learning

In [97]:
title_df['title'] = title_df['title'].apply(pre_process)
title_df['overview'] = title_df['overview'].apply(pre_process)
title_df['keywords2'] = title_df['keywords2'].apply(pre_process)
filmes_balanceados['title'] = filmes_balanceados2['title'].apply(pre_process)
filmes_balanceados['overview'] = filmes_balanceados2['overview'].apply(pre_process)
filmes_balanceados['keywords2'] = filmes_balanceados2['keywords2'].apply(pre_process)
df_upsampled['title'] = df_upsampled['title'].apply(pre_process)
df_upsampled['overview'] = df_upsampled['overview'].apply(pre_process)
df_upsampled['keywords2'] = df_upsampled['keywords2'].apply(pre_process)

#### Criação de matriz binária para conferencia de resultados 

In [105]:
# from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
# multilabel_binarizer.fit(title_df['genres'])

# transform target variable
y = multilabel_binarizer.fit_transform( title_df['genres']  )

# transform target variable
multilabel_binarizer2 = MultiLabelBinarizer()
y_b = multilabel_binarizer2.fit_transform(  filmes_balanceados2['genres2']    )

# y_b = multilabel_binarizer2.transform( filmes_balanceados2['genres'].unique()   )

# transform target variable
multilabel_binarizer3 = MultiLabelBinarizer()
y_b2 =multilabel_binarizer3.fit_transform(df_upsampled['genres2'])

# y_b2 = multilabel_binarizer3.transform([df_upsampled['genres'].to_list()])



In [106]:
[filmes_balanceados2['genres2'].values]

[array([list(['Action']), list(['Adventure']), list(['Fantasy']), ...,
        list(['Romance']), list(['TV Movie']), list(['Documentary'])],
       dtype=object)]

In [107]:
title_df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [108]:
multilabel_binarizer3.classes_

array(['Action', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music',
       'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller',
       'War', 'Western'], dtype=object)

In [109]:
multilabel_binarizer2.inverse_transform(y_b)

[('Action',),
 ('Adventure',),
 ('Fantasy',),
 ('Science Fiction',),
 ('Adventure',),
 ('Fantasy',),
 ('Action',),
 ('Action',),
 ('Adventure',),
 ('Crime',),
 ('Action',),
 ('Crime',),
 ('Drama',),
 ('Thriller',),
 ('Action',),
 ('Adventure',),
 ('Science Fiction',),
 ('Fantasy',),
 ('Action',),
 ('Adventure',),
 ('Animation',),
 ('Family',),
 ('Action',),
 ('Adventure',),
 ('Science Fiction',),
 ('Adventure',),
 ('Fantasy',),
 ('Family',),
 ('Action',),
 ('Adventure',),
 ('Fantasy',),
 ('Adventure',),
 ('Fantasy',),
 ('Action',),
 ('Science Fiction',),
 ('Adventure',),
 ('Action',),
 ('Thriller',),
 ('Crime',),
 ('Adventure',),
 ('Fantasy',),
 ('Action',),
 ('Action',),
 ('Adventure',),
 ('Western',),
 ('Action',),
 ('Adventure',),
 ('Fantasy',),
 ('Science Fiction',),
 ('Adventure',),
 ('Family',),
 ('Fantasy',),
 ('Science Fiction',),
 ('Action',),
 ('Adventure',),
 ('Adventure',),
 ('Action',),
 ('Fantasy',),
 ('Action',),
 ('Comedy',),
 ('Science Fiction',),
 ('Action',),
 ('Adve

In [110]:
y_b2.shape

(43643, 19)

#### Criação da matriz de termos x frequencia - Somente Título
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html

TfidfVectorizer -> Convert a collection of raw documents to a matrix of TF-IDF features.

In [111]:
xtrainb, xvalb, ytrainb, yvalb = model_selection.train_test_split(filmes_balanceados2['title'], y_b, test_size=0.2, random_state=512)


In [112]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.7, max_features=5000)
tfidf_vectorizer2 = TfidfVectorizer(max_df=0.7, max_features=5000)
tfidf_vectorizer3 = TfidfVectorizer(max_df=0.7, max_features=5000)
# from sklearn import model_selection
xtrain, xval, ytrain, yval = model_selection.train_test_split(title_df['title'], y, test_size=0.2, random_state=512)
xtrainb, xvalb, ytrainb, yvalb = model_selection.train_test_split(filmes_balanceados2['title'], y_b, test_size=0.2, random_state=512)
xtrainb2, xvalb2, ytrainb2, yvalb2 = model_selection.train_test_split(df_upsampled['title'], y_b2, test_size=0.2, random_state=512)

In [113]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

xtrain_tfidfb = tfidf_vectorizer2.fit_transform(xtrainb)
xval_tfidfb = tfidf_vectorizer2.transform(xvalb)

xtrain_tfidfb2 = tfidf_vectorizer3.fit_transform(xtrainb2)
xval_tfidfb2 = tfidf_vectorizer3.transform(xvalb2)

In [114]:
 multilabel_binarizer2.inverse_transform(y_b)

[('Action',),
 ('Adventure',),
 ('Fantasy',),
 ('Science Fiction',),
 ('Adventure',),
 ('Fantasy',),
 ('Action',),
 ('Action',),
 ('Adventure',),
 ('Crime',),
 ('Action',),
 ('Crime',),
 ('Drama',),
 ('Thriller',),
 ('Action',),
 ('Adventure',),
 ('Science Fiction',),
 ('Fantasy',),
 ('Action',),
 ('Adventure',),
 ('Animation',),
 ('Family',),
 ('Action',),
 ('Adventure',),
 ('Science Fiction',),
 ('Adventure',),
 ('Fantasy',),
 ('Family',),
 ('Action',),
 ('Adventure',),
 ('Fantasy',),
 ('Adventure',),
 ('Fantasy',),
 ('Action',),
 ('Science Fiction',),
 ('Adventure',),
 ('Action',),
 ('Thriller',),
 ('Crime',),
 ('Adventure',),
 ('Fantasy',),
 ('Action',),
 ('Action',),
 ('Adventure',),
 ('Western',),
 ('Action',),
 ('Adventure',),
 ('Fantasy',),
 ('Science Fiction',),
 ('Adventure',),
 ('Family',),
 ('Fantasy',),
 ('Science Fiction',),
 ('Action',),
 ('Adventure',),
 ('Adventure',),
 ('Action',),
 ('Fantasy',),
 ('Action',),
 ('Comedy',),
 ('Science Fiction',),
 ('Action',),
 ('Adve

In [115]:
def benchmark(clf, xtrain, xval, ytrain, yval):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(xtrain_tfidf, ytrain)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(xtrain_tfidf)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(yval, pred)
    print("accuracy:   %0.3f" % score)

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

        if opts.print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            for i, label in enumerate(target_names):
                top10 = np.argsort(clf.coef_[i])[-10:]
                print(trim("%s: %s" % (label, " ".join(feature_names[top10]))))
        print()

    if opts.print_report:
        print("classification report:")
        print(metrics.classification_report(yval, pred,
                                            target_names=target_names))

    if opts.print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(yval, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time

### Aplicação da Regressão Logistica

In [116]:
from sklearn.linear_model import LogisticRegression
# https://scikit-learn.org/stable/modules/model_evaluation.html
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics    
    
# Binary Relevance

from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score,average_precision_score,brier_score_loss
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from sklearn.metrics import multilabel_confusion_matrix, ndcg_score, precision_recall_curve
from sklearn.metrics import precision_recall_fscore_support, balanced_accuracy_score
# from sklearn.metrics import 


# accuracy_score 
# See the Regression metrics section of the user guide for further details.

# metrics.explained_variance_score(y_true, …)

# Explained variance regression score function

# metrics.max_error(y_true, y_pred)

# max_error metric calculates the maximum residual error.

# metrics.mean_absolute_error(y_true, y_pred, *)

# Mean absolute error regression loss

# metrics.mean_squared_error(y_true, y_pred, *)

# Mean squared error regression loss

# metrics.mean_squared_log_error(y_true, y_pred, *)

# Mean squared logarithmic error regression loss

# metrics.median_absolute_error(y_true, y_pred, *)

# Median absolute error regression loss

# metrics.r2_score(y_true, y_pred, *[, …])

# R^2 (coefficient of determination) regression score function.

# metrics.mean_poisson_deviance(y_true, y_pred, *)

# Mean Poisson deviance regression loss.

# metrics.mean_gamma_deviance(y_true, y_pred, *)

# Mean Gamma deviance regression loss.

# metrics.mean_tweedie_deviance(y_true, y_pred, *)

# Mean Tweedie deviance regression loss. 
# See the Multilabel ranking metrics section of the user guide for further details.

# metrics.coverage_error(y_true, y_score, *[, …])

# Coverage error measure

# metrics.label_ranking_average_precision_score(…)

# Compute ranking-based average precision

# metrics.label_ranking_loss(y_true, y_score, *)

# Compute Ranking loss measure

In [134]:

from sklearn.linear_model import LogisticRegression,SGDClassifier
#Sem tratamento de classes desbalanceadas
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

#Algoritmo penalizando as classes maiores
lr2 = LogisticRegression(class_weight = 'balanced')
clf2 = OneVsRestClassifier(lr2)

#Aqui as classes foram upsampled
lr3 = LogisticRegression(class_weight = 'balanced')
clf3 = OneVsRestClassifier(lr3)


In [135]:
clf.fit(xtrain_tfidf, ytrain)
clf2.fit(xtrain_tfidfb, ytrainb)
clf3.fit(xtrain_tfidfb2, ytrainb2)
clf4.fit(xtrain_tfidfb2, ytrainb2)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced', loss='log',
                                            penalty='l1'))

In [141]:
lr4 = SGDClassifier(shuffle=True, loss='log', penalty='l1', class_weight = 'balanced')
clf4 = OneVsRestClassifier(lr4)
clf4.fit(xtrain_tfidfb2, ytrainb2)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced', loss='log',
                                            penalty='l1'))

In [142]:
y_pred = clf.predict(xval_tfidf)
y_pred2 = clf2.predict(xval_tfidfb) 
y_pred3 = clf3.predict(xval_tfidfb2)
y_pred4 = clf4.predict(xval_tfidfb2)

In [145]:
from sklearn.metrics import precision_score
precision_score(yvalb2, y_pred4, average="micro")

0.31958676648960643

In [146]:
precision_score(yvalb2, y_pred3, average="micro")

0.36511943158977234

In [143]:
print('f1_score sem tratamento:'+str(f1_score(yval, y_pred, average="micro")))
print('f1_score penalizando as classes maiores:'+str(f1_score(yvalb, y_pred2, average="micro")))
print('f1_score com upsampled:'+str(f1_score(yvalb2, y_pred3, average="micro")))
print('f1_score com upsampled2:'+str(f1_score(yvalb2, y_pred4, average="micro")))

f1_score sem tratamento:0.22792607802874743
f1_score penalizando as classes maiores:0.1255109401298389
f1_score com upsampled:0.5186242613372286
f1_score com upsampled2:0.46822721980513515


In [124]:
y_pred_prob = clf.predict_proba(xval_tfidf)
y_pred_prob2 = clf2.predict_proba(xval_tfidfb)
y_pred_prob3 = clf3.predict_proba(xval_tfidfb2)

In [125]:
t = 0.4 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)
y_pred_new2 = (y_pred_prob2 >= t).astype(int)
y_pred_new3 = (y_pred_prob3 >= t).astype(int)

In [126]:
f1_score(yvalb2, y_pred_new3, average="micro")

0.4817203011126774

In [127]:
df_upsampled['genres'].unique()

array(['Drama', 'Comedy', 'Thriller', 'Action', 'Romance', 'Crime',
       'Science Fiction', 'Horror', 'Family', 'Fantasy', 'Mystery',
       'Animation', 'History', 'Music', 'War', 'Documentary', 'Western',
       'Foreign', 'TV Movie'], dtype=object)

In [38]:
def benchmark(clf, xtrain, xval, ytrain, yval,
              print_report=None, 
              print_top10=None, 
              feature_names=None,
              classes=None):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(xtrain, ytrain)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(xval)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = f1_score(yval, pred, average="micro")
    print("accuracy:   %0.3f" % score)
    print("classification report:")
    print(metrics.classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                            target_names=classes))
    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

         

#     if print_report:
        print("classification report:")
        print(metrics.classification_report(yval, pred,
#                                             labels=[1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
                                            target_names=classes))

#     if opts.print_cm:
#         print("confusion matrix:")
#         print(metrics.confusion_matrix(yval, pred))

#     print()
#     clf_descr = str(clf).split('(')[0]
#     return clf_descr, score, train_time, test_time

In [304]:
benchmark(clf, xtrain_tfidf, xval_tfidf, ytrain,
          yval,feature_names=tfidf_vectorizer.get_feature_names, 
         classes=multilabel_binarizer.classes_ 
         )

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=LogisticRegression())
train time: 0.505s
test time:  0.002s
accuracy:   0.228
classification report:
                 precision    recall  f1-score   support

         Action       0.75      0.07      0.13       217
      Adventure       0.75      0.02      0.04       146
      Animation       0.00      0.00      0.00        33
         Comedy       0.63      0.24      0.34       331
          Crime       0.00      0.00      0.00       135
    Documentary       0.00      0.00      0.00        24
          Drama       0.60      0.45      0.52       478
         Family       0.25      0.01      0.02        82
        Fantasy       0.00      0.00      0.00        74
        Foreign       0.00      0.00      0.00        11
        History       0.00      0.00      0.00        42
         Horror       1.00      0.03      0.05       116
          Music       0.00      0.0

In [305]:
benchmark(clf2, xtrain_tfidfb, xval_tfidfb, ytrainb,
          yvalb,  
         classes=multilabel_binarizer2.classes_
         )

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))
train time: 0.017s
test time:  0.001s
accuracy:   0.000
classification report:
                 precision    recall  f1-score   support

         Action       0.00      0.00      0.00         0
      Adventure       0.00      0.00      0.00         0
      Animation       0.00      0.00      0.00         0
         Comedy       0.00      0.00      0.00         0
          Crime       0.00      0.00      0.00         0
    Documentary       0.00      0.00      0.00         0
          Drama       0.00      0.00      0.00         0
         Family       0.00      0.00      0.00         0
        Fantasy       0.00      0.00      0.00         0
        Foreign       0.00      0.00      0.00         0
        History       0.00      0.00      0.00         0
         Horror       0.00      0.00      0.00         0
          Mu

In [300]:
benchmark(clf3, xtrain_tfidfb2, xval_tfidfb2, ytrainb2,
          yvalb2,feature_names=tfidf_vectorizer3.get_feature_names, 
         classes= multilabel_binarizer3.classes_ 
         )

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))
train time: 0.055s
test time:  0.002s
accuracy:   0.000
classification report:
                 precision    recall  f1-score   support

         Action       0.00      0.00      0.00         0
      Animation       0.00      0.00      0.00         0
         Comedy       0.00      0.00      0.00         0
          Crime       0.00      0.00      0.00         0
    Documentary       0.00      0.00      0.00         0
          Drama       0.00      0.00      0.00         0
         Family       0.00      0.00      0.00         0
        Fantasy       0.00      0.00      0.00         0
        Foreign       0.00      0.00      0.00         0
        History       0.00      0.00      0.00         0
         Horror       0.00      0.00      0.00         0
          Music       0.00      0.00      0.00         0
        Myst

In [146]:
def inference(text):
    processed_text = pre_process(text)
    q_vec = tfidf_vectorizer.transform([processed_text])
    q_pred = clf.predict(q_vec) 
    return multilabel_binarizer.inverse_transform(q_pred)
def inference2(text):
    processed_text = pre_process(text)
    q_vec = tfidf_vectorizer2.transform([processed_text])
    q_pred = clf2.predict(q_vec) 
    return multilabel_binarizer2.inverse_transform(q_pred)
def inference3(text):
    processed_text = pre_process(text)
    q_vec = tfidf_vectorizer3.transform([processed_text])
    q_pred = clf3.predict(q_vec) 
    return multilabel_binarizer3.inverse_transform(q_pred)
def CalculateAccuracy(y_test,pred_label):
    nnz = np.shape(y_test)[0] - np.count_nonzero(pred_label - y_test)
    acc = 100*nnz/float(np.shape(y_test)[0])
    return acc

In [149]:
y_pred3

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 1]])

In [150]:
CalculateAccuracy(yval, y_pred)

-134.75546305931323

In [151]:
xval.shape

(961,)

In [306]:
for i in range(10):
    k = xval.sample(1).index[0]
    print("Movie: ", title_df['title'][k],
          "\nPredicted genre: ", inference(xval[k])
         ), 
    print("Actual genre: ",title_df['genres'][k], "\n")

Movie:  Spider 
Predicted genre:  [('Action',)]
Actual genre:  ['Drama', 'Mystery', 'Thriller'] 

Movie:  St Vincent 
Predicted genre:  [('Drama',)]
Actual genre:  ['Comedy'] 

Movie:  The Man Who Knew Too Little 
Predicted genre:  [('Comedy',)]
Actual genre:  ['Comedy', 'Thriller', 'Crime', 'Action'] 

Movie:  Club Dread 
Predicted genre:  [('Drama',)]
Actual genre:  ['Comedy', 'Horror'] 

Movie:  The Departed 
Predicted genre:  [('Drama',)]
Actual genre:  ['Drama', 'Thriller', 'Crime'] 

Movie:  Shark Lake 
Predicted genre:  [()]
Actual genre:  ['Thriller'] 

Movie:  Insurgent 
Predicted genre:  [()]
Actual genre:  ['Adventure', 'Science Fiction', 'Thriller'] 

Movie:  Silverado 
Predicted genre:  [()]
Actual genre:  ['Action', 'Crime', 'Drama', 'Western'] 

Movie:  Romeo Must Die 
Predicted genre:  [('Thriller',)]
Actual genre:  ['Action', 'Crime', 'Thriller'] 

Movie:  About Time 
Predicted genre:  [('Comedy',)]
Actual genre:  ['Comedy', 'Drama', 'Science Fiction'] 



In [307]:
k = xvalb.sample(1).index[0] 
print(xvalb[k])
processed_text = pre_process(xvalb[k])
processed_text
q_vec = tfidf_vectorizer2.transform([processed_text])
q_vec
q_pred = clf2.predict(q_vec) 
q_pred
y_pred2
multilabel_binarizer3.inverse_transform(y_pred3) [3]
# return multilabel_binarizer2.inverse_transform(q_pred)
print(multilabel_binarizer3.inverse_transform(yvalb2) [3])
print(multilabel_binarizer3.inverse_transform(y_pred3) [3])
# print(inference2(xvalb[k]))

Rabbit-Proof Fence
()
()


In [310]:
for i in range(10):
    k = xvalb.sample(1).index[0]
    print("Movie: ", filmes_balanceados2['title'][k],
          "\nPredicted genre: ", inference2(xvalb[k])
         ), 
    print("Actual genre: ",filmes_balanceados2['genres'][k], "\n")

Movie:  The Prince 
Predicted genre:  [()]
Actual genre:  Action 

Movie:  Stranger Than Fiction 
Predicted genre:  [()]
Actual genre:  Romance 

Movie:  Undisputed 
Predicted genre:  [()]
Actual genre:  Drama 

Movie:  Iris 
Predicted genre:  [()]
Actual genre:  Romance 

Movie:  Elite Squad 
Predicted genre:  [()]
Actual genre:  Action 

Movie:  Collateral Damage 
Predicted genre:  [()]
Actual genre:  Action 

Movie:  Win a Date with Tad Hamilton! 
Predicted genre:  [()]
Actual genre:  Comedy 

Movie:  Black November 
Predicted genre:  [()]
Actual genre:  Action 

Movie:  Things to Do in Denver When You're Dead 
Predicted genre:  [()]
Actual genre:  Crime 

Movie:  Beverly Hills Cop 
Predicted genre:  [()]
Actual genre:  Crime 



In [311]:
for i in range(10):
    k = xvalb2.sample(1).index[0]
    print("Movie: ", df_upsampled['title'][k],
          "\nPredicted genre: ", inference3(xvalb2[k])
         ), 
    print("Actual genre: ",df_upsampled['genres'][k], "\n")

Movie:  5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
5100    The Right Stuff
Name: title, dtype: object 
Predicted genre:  [()]
Actual genre:  5100    History
5100    History
5100    History
5100    History
5100    History
5100    History
5100    History
5100    History
5100    History
5100    History
5100    History
Name: genres, dtype: object 

Movie:  12158    Signed Sealed Delivered
12158    Signed Sealed Delivered
12158    Signed Sealed Delivered
12158    Signed Sealed Delivered
12158    Signed Sealed Delivered
                  ...           
12158    Signed Sealed Delivered
12158    Signed Sealed Delivered
12158    Signed Sealed Delivered
12158    Signed Sealed Delivered
12158    Signed Sealed Delivered
Name: title, Length: 296, dtype: object 
Predicted genre:  [()]
Actual genre:  12158 

In [1]:
benchmark(clf, xtrain_tfidf, xval_tfidf, ytrain, yval)

NameError: name 'benchmark' is not defined

In [167]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt
results = []
for clf, name in ( 
        (Perceptron(max_iter=50), "Perceptron") ):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

for penalty in ["l2", "l1"]:
    print('=' * 80)
    print("%s penalty" % penalty.upper())
    # Train Liblinear model
    results.append(benchmark(LinearSVC(penalty=penalty, dual=False,
                                       tol=1e-3)))

    # Train SGD model
    results.append(benchmark(SGDClassifier(alpha=.0001, max_iter=50,
                                           penalty=penalty)))

# Train SGD with Elastic Net penalty
print('=' * 80)
print("Elastic-Net penalty")
results.append(benchmark(SGDClassifier(alpha=.0001, max_iter=50,
                                       penalty="elasticnet")))

# Train NearestCentroid without threshold
print('=' * 80)
print("NearestCentroid (aka Rocchio classifier)")
results.append(benchmark(NearestCentroid()))

# Train sparse Naive Bayes classifiers
print('=' * 80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=.01)))
results.append(benchmark(BernoulliNB(alpha=.01)))
results.append(benchmark(ComplementNB(alpha=.1)))

print('=' * 80)
print("LinearSVC with L1-based feature selection")
# The smaller C, the stronger the regularization.
# The more regularization, the more sparsity.
results.append(benchmark(Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False,
                                                  tol=1e-3))),
  ('classification', LinearSVC(penalty="l2"))])))

TypeError: cannot unpack non-iterable Perceptron object